In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_iris

In [3]:
iris = load_iris()
x = iris.data
y = iris.target

In [4]:
train_x, train_y =x, y

In [5]:
x.shape, y.shape

((150, 4), (150,))

In [6]:
import tensorflow as tf
x = tf.placeholder(tf.float32, shape=(150, 4))
y = tf.placeholder(tf.int32,shape = (150,))

with tf.Session() as sess:
    sess.run(x, feed_dict={x: iris.data, y:iris.target})  # Will succeed.


/home/odemakinde/cd/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
var_x = tf.Variable(iris.data, dtype = tf.float32)
var_y = tf.Variable(iris.target, dtype = tf.float32)

In [8]:
help (tf.Variable)

Help on class Variable in module tensorflow.python.ops.variables:

class Variable(builtins.object)
 |  See the @{$variables$Variables How To} for a high level overview.
 |  
 |  A variable maintains state in the graph across calls to `run()`. You add a
 |  variable to the graph by constructing an instance of the class `Variable`.
 |  
 |  The `Variable()` constructor requires an initial value for the variable,
 |  which can be a `Tensor` of any type and shape. The initial value defines the
 |  type and shape of the variable. After construction, the type and shape of
 |  the variable are fixed. The value can be changed using one of the assign
 |  methods.
 |  
 |  If you want to change the shape of a variable later you have to use an
 |  `assign` Op with `validate_shape=False`.
 |  
 |  Just like any `Tensor`, variables created with `Variable()` can be used as
 |  inputs for other Ops in the graph. Additionally, all the operators
 |  overloaded for the `Tensor` class are carried over to

In [9]:
# implementing batch normalizatioin

n_inputs = 4
n_hidden1 = 12
n_hidden2 = 24
n_outputs = 10

x = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'x')

training = tf.placeholder_with_default(False, shape = (), name = 'training')

hidden1 = tf.layers.dense(x, n_hidden1, name = 'hidden1')
bn1 = tf.layers.batch_normalization(hidden1, training = training, momentum = 0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name = 'hidden2')
bn2 = tf.layers.batch_normalization(hidden2, training = training, momentum = 0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name = 'outputs')
logits = tf.layers.batch_normalization(logits_before_bn, training = training, momentum = 0.9)


In [10]:
from functools import partial

In [11]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
epochs = 10

test_y = 1.0

init = tf.global_variables_initializer()
saver = tf.train.Saver()


with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits= logits)
    loss = tf.reduce_mean(xentropy, name = 'loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
training_op = optimizer.minimize(loss)

with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
        sess.run([training_op, extra_update_ops],
                feed_dict = {training: True, x: train_x, y: train_y})
        correct = tf.nn.in_top_k(logits, train_y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        accuracy_val = accuracy.eval(feed_dict = {x:train_x, y:train_y})
        print(epoch, 'Train accuracy:', accuracy_val)
    save_path = saver.save(sess, './my_model_final.ckpt')

0 Train accuracy: 0.0
1 Train accuracy: 0.0
2 Train accuracy: 0.0
3 Train accuracy: 0.0
4 Train accuracy: 0.013333334
5 Train accuracy: 0.046666667
6 Train accuracy: 0.18
7 Train accuracy: 0.23333333
8 Train accuracy: 0.24666667
9 Train accuracy: 0.25333333
